<h5><b>Install Package and Library</h5></b>

In [1]:
# Install package
!pip install selenium
!pip install webdriver-manager

# Install library
import datetime as dt
import numpy as np
import os
import pandas as pd
import time
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

# Install wget
!apt-get install wget

# Download the chromedriver.exe from GitHub
os.system('wget https://github.com/SamapanThongmee/Thai-Stock-Tickers-from-web-scrapping/blob/main/chromedriver.exe')

# Create function
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


<h5><b>Scrap SET50 Index Futures</b></h5>

In [3]:
def getFutures(symbol='S50U23'):
    futures = pd.DataFrame(columns=['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI'])

    try:
        for page in range(15):
            url = f'https://www.tfex.co.th/tfex/historicalTrading.html?symbol={symbol}&series=&page={page}&periodView=A&locale=en_US'
            driver.get(url)
            data = driver.page_source
            scrap = pd.read_html(data)[0]
            mask = scrap['Date'] == 'Grand Total'
            indices = int(scrap.index[mask][0])
            futures_data = scrap.iloc[:indices, :][['Date', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']].replace('-', 0)
            futures_data['Date'] = pd.to_datetime(futures_data['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
            futures_data['SP'] = pd.to_numeric(futures_data['SP'])
            futures_data['Vol'] = pd.to_numeric(futures_data['Vol'])
            futures_data['OI'] = pd.to_numeric(futures_data['OI'])
            futures_data['Symbol'] = symbol
            futures = pd.concat([futures, futures_data], axis=0)
            futures = futures.drop_duplicates(subset=['Date'], keep='last')

    except Exception as e:
        print(f"An error occurred: {e}")

    futures = futures[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'SP', 'Vol', 'OI']]
    futures = futures.sort_values(by='Date').drop_duplicates(subset='Date', keep='first').reset_index(drop=True)

    return futures

In [4]:
# Define a list of symbols
symbols = ['S50M', 'S50H', 'S50U', 'S50Z']

# Define a range of years
years = range(6, 24)

# Initialize an empty DataFrame to store the futures data
futures_df = pd.DataFrame()

# Loop through the symbols and years to fetch the data
for symbol in symbols:
    for year in years:
        futures_key = f"{symbol}{year:02d}"
        futures_data = getFutures(symbol=futures_key)
        futures_data_df = pd.DataFrame(futures_data)
        futures_data_df['Symbol'] = futures_key
        futures_df = futures_df.append(futures_data_df, ignore_index=True)

futures_df

An error occurred: No tables found


,Date,Symbol,Open,High,Low,Close,SP,Vol,OI
0,2006-04-28,S50M06,536.5,542.0,535.6,536.2,536.0,152,98.0
1,2006-05-02,S50M06,536.0,537.2,536.0,536.1,536.1,53,114.0
2,2006-05-03,S50M06,537.0,538.5,537.0,538.5,538.5,75,132.0
3,2006-05-04,S50M06,536.8,536.8,535.6,535.7,535.7,81,166.0
4,2006-05-08,S50M06,536.5,542.0,536.5,541.0,541.0,109,165.0
...,...,...,...,...,...,...,...,...,...
16886,2023-10-27,S50Z23,853.0,863.3,847.7,858.8,859.1,289834,614849.0
16887,2023-10-30,S50Z23,856.0,869.4,853.5,868.4,868.3,260213,620138.0
16888,2023-10-31,S50Z23,869.0,869.0,852.8,858.9,859.0,259224,604517.0
16889,2023-11-01,S50Z23,859.0,862.7,849.2,854.6,855.0,278113,621977.0


<h5><b>Export to .csv</b></h5>

In [5]:
futures_df.to_csv('SET50Futures.csv')